In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**LOADING THE EFFICIENTNET MODEL DATASET TO CURRENT SYS**

In [ ]:
paths=["../input/pytorch-image-models/pytorch-image-models-master","../input/image-fmix/FMix-master"]
base_dir="../input/cassava-leaf-disease-classification"
import sys
for pat in paths:
    sys.path.append(pat)

In [ ]:
from fmix import sample_mask,binarise_mask,make_low_freq_image

In [ ]:
import timm
import os
import glob as glob
import cv2
import skimage as io
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time 
from datetime import datetime
import random
from tqdm import tqdm,tqdm_notebook
tqdm.pandas()

import sklearn
from sklearn import metrics
from sklearn.model_selection import GroupKFold,StratifiedKFold
from sklearn.metrics import roc_auc_score,log_loss

import pydicom
import joblib
import warnings
import json
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler,RandomSampler
from torch.cuda.amp import autocast,GradScaler

from scipy.ndimage.interpolation import zoom
from torch.nn.modules.loss import _WeightedLoss


In [ ]:
CFG = {
    'fold_num': 5,
    'seed': 719,
    'model_arch': 'tf_efficientnet_b4_ns',
    'img_size': 512,
    'epochs': 10,
    'train_bs': 16,
    'valid_bs': 32,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 4,
    'accum_iter': 2,
    'verbose_step': 1,
    'device': 'cuda:0'
}

**LOADING THE TRAINING CSV FILE**

In [ ]:
train=pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
train.head()

In [ ]:
train.info

In [ ]:
train.label.value_counts()

**LOADING THE SUBMISSION CSV FILE**

In [ ]:
submission=pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
submission

In [ ]:
def seeder(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark=True
    torch.backends.cudnn.deterministic=True
    os.environ["PYTHONHASHSEED"]=str(seed)

In [ ]:
def get_img(path):
    image=cv2.imread(path)
    image=image[:,:,::-1]
    return image
plt.figure(figsize=(15,15))
plt.imshow(get_img('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg'))
plt.title('1000015157.jpg')
plt.axis('off')
plt.show()

In [ ]:
def rand_bbox(size,lam):
    W=size[0]
    H=size[1]
    cut_rat=np.sqrt(1.-lam)
    cut_w=np.int(W*cut_rat)
    cut_h=np.int(H*cut_rat)
    
    c_x=np.random.randint(W)
    c_y=np.random.randint(H)
    
    bbx1=np.clip((c_x-cut_w)//2,0,W)
    bbx2=np.clip((c_x+cut_w)//2,0,W)
    
    bby1=np.clip((c_y-cut_h)//2,0,H)
    bby2=np.clip((c_y+cut_h)//2,0,H)
    
    return bbx1,bby1,bbx2,bby2

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, df, data_root, 
                 transforms=None, 
                 output_label=True, 
                 one_hot_label=False,
                 do_fmix=False, 
                 fmix_params={
                     'alpha': 1., 
                     'decay_power': 3., 
                     'shape': (CFG['img_size'], CFG['img_size']),
                     'max_soft': True, 
                     'reformulate': False
                 },
                 do_cutmix=False,
                 cutmix_params={
                     'alpha': 1,
                 }
                ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.do_fmix = do_fmix
        self.fmix_params = fmix_params
        self.do_cutmix = do_cutmix
        self.cutmix_params = cutmix_params
        
        self.output_label = output_label
        self.one_hot_label = one_hot_label
        
        if output_label == True:
            self.labels = self.df['label'].values
            
            
            if one_hot_label is True:
                self.labels = np.eye(self.df['label'].max()+1)[self.labels]
            
    def __len__(self):
        return self.df.shape[0] # return len(df.shape) also works
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.labels[index]
          
        img=get_img("{}/{}".format(self.data_root, self.df.loc[index]['image_id']))
        

        if self.transforms:
            img = self.transforms(image=img)
            img = img["image"]
        
        if self.do_fmix and np.random.uniform(0., 1., size=1)[0] > 0.5:
            with torch.no_grad():
                
                lam = np.clip(np.random.beta(self.fmix_params['alpha'], self.fmix_params['alpha']),0.6,0.7)
                
                mask = make_low_freq_image(self.fmix_params['decay_power'], self.fmix_params['shape'])
                mask = binarise_mask(mask, lam, self.fmix_params['shape'], self.fmix_params['max_soft'])
    
                fmix_ix = np.random.choice(self.df.index, size=1)[0]
                fmix_img=get_img("{}/{}".format(self.data_root, self.df.iloc[fmix_ix]['image_id']))

                if self.transforms:
                    fmix_img = self.transforms(image=fmix_img)["image"]

                mask_torch = torch.from_numpy(mask)
                
                img = mask_torch*img+(1.-mask_torch)*fmix_img

                rate = mask.sum()/CFG['img_size']/CFG['img_size']
                target = rate*target + (1.-rate)*self.labels[fmix_ix]
                
        if self.do_cutmix and np.random.uniform(0., 1., size=1)[0] > 0.5:
            with torch.no_grad():
                
                lam = np.clip(np.random.beta(self.cutmix_params['alpha'], self.cutmix_params['alpha']),0.3,0.4)                
                cutmix_idx = np.random.choice(self.df.index, size=1)[0]
                cutmix_img=get_img("{}/{}".format(self.data_root, self.df.iloc[cutmix_idx]['image_id']))
                
                if self.transforms:
                    cutmix_img = self.transforms(image=cutmix_img)["image"]

                
                bbx1,bby1,bbx2,bby2=rand_bbox((CFG['img_size'],CFG['img_size']),lam)
                img[:,bbx1:bbx2,bby1:bby2]=cutmix_img[:,bbx1:bbx2,bby1:bby2]
                rate=1-((bbx2-bbx1)*(bby2-bby1))/(CFG['img_size']*CFG['img_size'])
                
                target=rate*target+(1-rate)*self.labels[cutmix_idx]
                
        if self.output_label==True:
            return img,torch.tensor(target,dtype=torch.double)
        else:
            return img
                



                    
                

        



In [ ]:
from albumentations import (HorizontalFlip,
                            VerticalFlip,
                            CLAHE,
                            ShiftScaleRotate,
                            IAAPerspective,
                            IAAAdditiveGaussianNoise,
                            GaussNoise,
                            IAAPiecewiseAffine,
                            IAASharpen,
                            IAAEmboss,
                            RandomRotate90,
                            Blur,
                            MotionBlur,
                            MedianBlur,
                            HueSaturationValue,
                            Transpose,
                            OpticalDistortion,
                            GridDistortion,
                            RandomResizedCrop,
                            RandomBrightnessContrast,
                            CoarseDropout,
                            Resize,
                            Flip,
                            OneOf,
                            Compose,
                            Cutout,
                            Normalize,
                            CenterCrop
                            )


from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_valid_transforms():
    return Compose([
            CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)


In [ ]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x


In [ ]:
def prepare_dataloader(df,trn_idx,val_idx,data_root="../input/cassava-leaf-disease-classification/train_images"):
    
    _train=df.loc[trn_idx,:].reset_index(drop=True)
    _valid=df.loc[val_idx,:].reset_index(drop=True)
    
    train_ds=CassavaDataset(_train,
                            data_root,
                            transforms=get_train_transforms(),
                            output_label=True,
                            one_hot_label=False,
                            do_fmix=True,
                            do_cutmix=True)
    
    valid_ds=CassavaDataset(_valid,
                            data_root,
                            transforms=get_valid_transforms(),
                            output_label=True)
    
    train_loader=torch.utils.data.DataLoader(train_ds,
                                             shuffle=True,
                                             batch_size=CFG["train_bs"],
                                             pin_memory=False,
                                             drop_last=False,
                                             num_workers=CFG["num_workers"])
    
    valid_loader=torch.utils.data.DataLoader(valid_ds,
                                             shuffle=False,
                                             batch_size=CFG["valid_bs"],
                                             pin_memory=False,
                                             num_workers=CFG["num_workers"])
    
    return train_loader,valid_loader



    

In [ ]:
def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device, scheduler=None, schd_batch_update=False):
    model.train()

    t = time.time()
    running_loss = None

    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        #print("Input shape")
        #print(image_labels.shape,imgs.shape)
        with autocast():
            image_preds = model(imgs)   #output = model(input)
            #print("OUTput shape",image_preds.shape)

            loss = loss_fn(image_preds, image_labels)
            
            scaler.scale(loss).backward()

            if running_loss is None:
                running_loss = loss.item()
            else:
                running_loss = running_loss * .99 + loss.item() * .01

            if ((step + 1) %  CFG['accum_iter'] == 0) or ((step + 1) == len(train_loader)):
                # may unscale_ here if desired (e.g., to allow clipping unscaled gradients)

                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad() 
                
                if scheduler is not None and schd_batch_update:
                    scheduler.step()

            if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(train_loader)):
                description = f'epoch {epoch} loss: {running_loss:.4f}'
                
                pbar.set_description(description)
                
    if scheduler is not None and not schd_batch_update:
        scheduler.step()


In [ ]:
def valid_one_epoch(epoch, model, loss_fn, valid_loader, device, scheduler=None, schd_loss_update=False):
    model.eval()

    t = time.time()
    
    loss_sum = 0
    sample_num = 0
    image_preds_all = []
    image_targets_all = []
    
    pbar = tqdm(enumerate(valid_loader), total=len(valid_loader))
    for step, (imgs, image_labels) in pbar:
        
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        image_preds = model(imgs)
        
        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]        

        
        loss = loss_fn(image_preds, image_labels)
        
        loss_sum += loss.item()*image_labels.shape[0]
        sample_num += image_labels.shape[0]  

        if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(val_loader)):
            description = f'epoch {epoch} loss: {loss_sum/sample_num:.4f}'
            pbar.set_description(description)
    
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    print('Multiclass VAlidation Accuracy = {:.4f}'.format((image_preds_all==image_targets_all).mean()))
    
    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()
                





In [ ]:
class MyCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean'):
        super().__init__(weight=weight, reduction=reduction)
        self.weight = weight
        self.reduction = reduction

    def forward(self, inputs, targets):
        lsm = F.log_softmax(inputs, -1)

        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)

        loss = -(targets * lsm).sum(-1)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss


In [ ]:
if __name__ == '__main__':

    seeder(CFG['seed'])
    
    sfr = StratifiedKFold(n_splits=CFG['fold_num'], shuffle=True, random_state=CFG['seed'])
    folds = sfr.split(np.arange(train.shape[0]), train.label.values)
    
    for fold, (trn_idx, val_idx) in enumerate(folds):

        if fold > 0:
            break 

        print('Training started with {}th fold'.format(fold))

        print(len(trn_idx), len(val_idx))
        train_loader, val_loader = prepare_dataloader(train, trn_idx, val_idx, data_root='../input/cassava-leaf-disease-classification/train_images/')

        device = torch.device(CFG['device'])
        
        model = CassvaImgClassifier(CFG['model_arch'], train.label.nunique(), pretrained=True).to(device)
        
        scaler = GradScaler()   
        
        optimizer = torch.optim.Adam(model.parameters(), lr=CFG['lr'], weight_decay=CFG['weight_decay'])
        
        #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=CFG['epochs']-1)
        
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=CFG['T_0'], T_mult=1, eta_min=CFG['min_lr'], last_epoch=-1)
        
        #scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=25,max_lr=CFG['lr'], epochs=CFG['epochs'], steps_per_epoch=len(train_loader))
        
        loss_tr = nn.CrossEntropyLoss().to(device)
        loss_vl = nn.CrossEntropyLoss().to(device)
    
        for epoch in range(CFG['epochs']):
            train_one_epoch(epoch, model, loss_tr, optimizer, train_loader, device, scheduler=scheduler, schd_batch_update=False)

            with torch.no_grad():
                valid_one_epoch(epoch, model, loss_vl, val_loader, device, scheduler=None, schd_loss_update=False)

            #torch.save(model.state_dict(),'{}_fold_{}_{}'.format(CFG['model_arch'], fold, epoch))
            
        #torch.save(model.cnn_model.state_dict(),'{}/cnn_model_fold_{}_{}'.format(CFG['model_path'], fold, CFG['tag']))
        
        #del model, optimizer, train_loader, val_loader, scaler, scheduler
        #torch.cuda.empty_cache()


                


           
        


In [ ]:
test_ds=CassavaDataset(submission, data_root="../input/cassava-leaf-disease-classification/test_images", 
                       transforms=get_valid_transforms(),
                       output_label=False, 
                       one_hot_label=False,
                       do_fmix=False, 
                       fmix_params={
                       'alpha': 1., 
                       'decay_power': 3., 
                       'shape': (CFG['img_size'], CFG['img_size']),
                       'max_soft': True, 
                       'reformulate': False},
                       do_cutmix=False,
                       cutmix_params={'alpha': 1}
                       )
test_loader=torch.utils.data.DataLoader(test_ds,
                                        shuffle=False,
                                        batch_size=CFG["train_bs"],
                                        pin_memory=False,
                                        drop_last=False,
                                        num_workers=CFG["num_workers"])

def pred_output(model,test_loader,device=CFG["device"]):
    img_preds_all=[]
    for img in test_loader:
        img=img.to(device).float()
        img_preds=model(img)
        img_preds_all += [torch.argmax(img_preds, 1).detach().cpu().numpy()]
    img_preds_all=np.concatenate(img_preds_all)
    return img_preds_all

In [ ]:
submission_df=pd.DataFrame({"image_id":submission.image_id.values.tolist(),"label":pred_output(model,test_loader).tolist()})
submission_df

In [ ]:
submission_df.to_csv("submission.csv",index=False)